In [1]:
!pip install caer canaro

In [2]:
import os
import caer
import numpy as np
import cv2 as cv

import gc


In [6]:
IMG_SIZE = (80, 80)
channels = 1
char_path = r'../input/the-simpsons-characters-dataset/simpsons_dataset'

## Procedimiento
* grab the top 10 character, which have the most number of images for class






In [12]:
char_dict = {}
# grab the amount of images per class
for char in os.listdir(char_path):
    char_dict[char] = len(os.listdir(os.path.join(char_path,char)))
    
# Sort in descending order
char_dict = caer.sort_dict(char_dict, descending=True)

char_dict

In [13]:
# en este punto la lista ya esta ordenada
characters = []
count = 0
for i in char_dict:
    characters.append(i[0])
    count += 1
    if count >= 10:
        break
        
characters
        
    

In [14]:
# Create the training data

train = caer.preprocess_from_dir(char_path, characters, channels=channels, IMG_SIZE=IMG_SIZE, isShuffle=True)


In [16]:
# how many images in the training set?

len(train)

In [17]:
import matplotlib.pyplot as plt


plt.figure(figsize=(30,30))
plt.imshow(train[0][0], cmap='gray')
plt.show()


In [18]:
# separate training set en features, label, and reshape into a 4 dimension tensor 
featureSet, labels = caer.sep_train(train, IMG_SIZE= IMG_SIZE)

In [19]:
# Normalize the featureSet==>(0,1)
# reason  the network will be able to learn features  much faster 
# necesitamos one hot encoded en las labels
from tensorflow.keras.utils import to_categorical
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(characters))

In [20]:
X_train, X_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=.2)

In [31]:
BATCH_SIZE = 32
EPOCHS = 10

In [23]:
import canaro
# Image data genetor
# synthesize new images from already existing images to help introduce some randomness to our network
# and make it perform better(data augmentation)

datagen = canaro.generators.imageDataGenerator() 
train_gen = datagen.flow(X_train, y_train, batch_size=BATCH_SIZE)

In [26]:
# Creating the model

model = canaro.models.createSimpsonsModel(IMG_SIZE=IMG_SIZE, channels=channels, output_dim=len(characters),
                                         loss='binary_crossentropy', decay=1e-6, learning_rate=0.001,
                                         momentum=0.9, nesterov=True)

In [27]:

model.summary()

In [29]:
from tensorflow.keras.callbacks import LearningRateScheduler
callbacks_list = [LearningRateScheduler(canaro.lr_schedule)]


In [33]:
# Train a model

training = model.fit(train_gen,
                    steps_per_epoch=len(X_train)//BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_val, y_val),
                    validation_steps=len(y_val)//BATCH_SIZE,
                    callbacks= callbacks_list)


In [34]:
characters

In [51]:
test_path = r'../input/the-simpsons-characters-dataset/kaggle_simpson_testset/kaggle_simpson_testset/charles_montgomery_burns_0.jpg'

In [57]:
img = cv.imread(test_path)
plt.imshow(img, cmap='gray')
plt.show()

In [41]:
def prepare(img):
    import cv2 as cv
    img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)
    img = cv.resize(img, IMG_SIZE)
    img = caer.reshape(img, IMG_SIZE, 1)
    return img

In [58]:
predictions = model.predict(prepare(img))

In [59]:
predictions

In [60]:
print(characters[np.argmax(predictions[0])])